# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
import random
random.seed(0)
import warnings
warnings.filterwarnings("ignore")

In [2]:
from keras.backend import backend
from keras.datasets import cifar10

(x_train, y_train_num), (x_test, y_test_num) = cifar10.load_data()

Using TensorFlow backend.


In [3]:
print ("x_train:", x_train.shape)
print ("y_train:",y_train_num.shape)
print ("x_test:", x_test.shape)
print ("y_test:", y_test_num.shape)

x_train: (50000, 32, 32, 3)
y_train: (50000, 1)
x_test: (10000, 32, 32, 3)
y_test: (10000, 1)


In [0]:
import numpy as np
X_train_0_4 = x_train[np.where(y_train_num < 5)[0]]
y_train_0_4 = y_train_num[np.where(y_train_num < 5)[0]]
X_test_0_4 = x_test[np.where(y_test_num < 5)[0]]
y_test_0_4 = y_test_num[np.where(y_test_num < 5)[0]]

X_train_5_9 = x_train[np.where(y_train_num >= 5)[0]]
y_train_5_9 = y_train_num[np.where(y_train_num >= 5)[0]]-5
X_test_5_9 = x_test[np.where(y_test_num >= 5)[0]]
y_test_5_9 = y_test_num[np.where(y_test_num >= 5)[0]]-5

In [5]:
print("X_train__0_4:", X_train_0_4.shape)
print("y_train__0_4:", y_train_0_4.shape)
print("X_test__0_4:", X_test_0_4.shape)
print("y_test__0_4:", y_test_0_4.shape)
print("X_train__5_9:", X_train_5_9.shape)
print("y_train__5_9:", y_train_5_9.shape)
print("X_test__5_9:", X_test_5_9.shape)
print("y_test__5_9:", y_test_5_9.shape)

X_train__0_4: (25000, 32, 32, 3)
y_train__0_4: (25000, 1)
X_test__0_4: (5000, 32, 32, 3)
y_test__0_4: (5000, 1)
X_train__5_9: (25000, 32, 32, 3)
y_train__5_9: (25000, 1)
X_test__5_9: (5000, 32, 32, 3)
y_test__5_9: (5000, 1)


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
X_train_0_4 = X_train_0_4.astype('float32')
X_test_0_4 = X_test_0_4.astype('float32')
X_train_5_9 = X_train_5_9.astype('float32')
X_test_5_9 = X_test_5_9.astype('float32')

X_train_0_4 /= 255
X_test_0_4  /= 255
X_train_5_9 /= 255
X_test_5_9 /= 255

import keras
y_train_0_4 = keras.utils.to_categorical(y_train_0_4, 5)
y_train_5_9 = keras.utils.to_categorical(y_train_5_9, 5)
y_test_0_4 = keras.utils.to_categorical(y_test_0_4, 5)
y_test_5_9 = keras.utils.to_categorical(y_test_5_9, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32,32,3),name='conv_1'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',name='conv_2'))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu',name='conv_3'))
model.add(Conv2D(256, kernel_size=(4, 4), activation='relu',name='conv_4'))
model.add(Conv2D(128, kernel_size=(4, 4), activation='relu',name='conv_5'))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu',name='conv_6'))
model.add(MaxPooling2D(pool_size=(2, 2),name='max_1'))
model.add(Dropout(0.2,name='drop_1'))

In [0]:
model.add(Flatten())
model.add(Dense(128, activation='relu',name='dense_1'))
model.add(Dropout(0.2,name='drop_2'))
model.add(Dense(5, activation='softmax',name='dense_3'))

In [43]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
optimizer = Adam(lr=0.001)
model.compile(loss=categorical_crossentropy,
              optimizer= optimizer,
              metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
model.fit(X_train_0_4, y_train_0_4,
          batch_size= 256,
          epochs=100,
          callbacks=[callback],
          verbose=1,
          validation_data=(X_test_0_4, y_test_0_4))

Train on 25000 samples, validate on 5000 samples
Epoch 1/100
25000/25000 [==============================] - 10s 397us/step - loss: 1.2063 - acc: 0.4817 - val_loss: 0.9645 - val_acc: 0.6094
Epoch 2/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.9378 - acc: 0.6236 - val_loss: 0.8667 - val_acc: 0.6574
Epoch 3/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.8432 - acc: 0.6674 - val_loss: 0.8110 - val_acc: 0.6836
Epoch 4/100
25000/25000 [==============================] - 7s 294us/step - loss: 0.7511 - acc: 0.7104 - val_loss: 0.7602 - val_acc: 0.7068
Epoch 5/100
25000/25000 [==============================] - 7s 295us/step - loss: 0.6903 - acc: 0.7376 - val_loss: 0.6652 - val_acc: 0.7514
Epoch 6/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.6342 - acc: 0.7616 - val_loss: 0.6690 - val_acc: 0.7460
Epoch 7/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.5732 - acc: 0.7824 - val_loss: 0.

In [44]:
score = model.evaluate(X_test_0_4, y_test_0_4)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5000/5000 [==============================] - 1s 207us/step
Test loss: 0.5914711172103881
Test accuracy: 0.8014


In [46]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model.save('./cnn_cifar10_4.h5')
model.save_weights('./cnn_cifar10_4_weights.h5')

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [49]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

conv_1
False
conv_2
False
conv_3
False
conv_4
False
conv_5
False
conv_6
False
max_1
False
drop_1
False
flatten_8
False
dense_1
True
drop_2
False
dense_3
True


### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [50]:
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
model.fit(X_train_5_9, y_train_5_9,
          batch_size= 256,
          epochs=100,
          callbacks=[callback],
          verbose=1,
          validation_data=(X_test_5_9, y_test_5_9))

Train on 25000 samples, validate on 5000 samples
Epoch 1/100
25000/25000 [==============================] - 7s 298us/step - loss: 1.2666 - acc: 0.5030 - val_loss: 0.7600 - val_acc: 0.7258
Epoch 2/100
25000/25000 [==============================] - 7s 297us/step - loss: 0.6857 - acc: 0.7500 - val_loss: 0.5003 - val_acc: 0.8154
Epoch 3/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.5277 - acc: 0.8090 - val_loss: 0.4614 - val_acc: 0.8302
Epoch 4/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.4530 - acc: 0.8389 - val_loss: 0.4656 - val_acc: 0.8288
Epoch 5/100
25000/25000 [==============================] - 7s 297us/step - loss: 0.4147 - acc: 0.8523 - val_loss: 0.3737 - val_acc: 0.8652
Epoch 6/100
25000/25000 [==============================] - 7s 298us/step - loss: 0.3570 - acc: 0.8730 - val_loss: 0.3716 - val_acc: 0.8662
Epoch 7/100
25000/25000 [==============================] - 7s 296us/step - loss: 0.3246 - acc: 0.8832 - val_loss: 0.3

In [54]:
score = model.evaluate(X_test_5_9, y_test_5_9)
print('Test loss:', score[0])
print('Test accuracy for the classes 5 to 9 ( Transfer Learning from the classs 0 to 4):', score[1])

5000/5000 [==============================] - 1s 206us/step
Test loss: 0.3809063903808594
Test accuracy for the classes 5 to 9 ( Transfer Learning from the classs 0 to 4): 0.8762


# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [57]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [59]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [60]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [61]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(twenty_train.data)
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)
# encode train document
vector_trainX = vectorizer.transform(twenty_train.data)

vectorizer.fit(twenty_train.data)
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)
# encode train document
vector_testX = vectorizer.transform(twenty_test.data)


# summarize encoded vector
print(vector_trainX.shape)
print(vector_testX.shape)
# print(vector.toarray())

(2257, 35788)
(1502, 35788)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression
model_logistic = LogisticRegression()

In [68]:
model_logistic.fit(vector_trainX, twenty_train.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
y_predict_logistic_test = model_logistic.predict(vector_testX)
y_predict_logistic_train = model_logistic.predict(vector_trainX)

In [73]:
from sklearn import metrics
print("Train accuracy score:", metrics.accuracy_score(twenty_train.target,y_predict_logistic_train))
print("Test accuracy score:", metrics.accuracy_score(twenty_test.target,y_predict_logistic_test))

Train accuracy score: 0.9827204253433761
Test accuracy score: 0.8868175765645806
